In [ ]:
#%%
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",  one_hot=True)
global_step = tf.Variable(0, trainable=False, name='global_step')       

#변수들
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)                  #
W1 = tf.get_variable("W1", shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable("W2", shape=[256, 50], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable("W3", shape=[50, 10], initializer=tf.contrib.layers.xavier_initializer())

b1= tf.Variable(tf.random_normal([256]))
b2= tf.Variable(tf.random_normal([50]))
b3= tf.Variable(tf.random_normal([10]))

#모델
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)         #
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)         #
L3 = tf.matmul(L2, W3) + b3
model = L3
cost =   tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))                        
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost) 

#검증을 위한 코드/모니터링
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
tf.summary.scalar('cost', cost)                     
tf.summary.scalar('accuracy', accuracy)    
tf.summary.histogram("W1", W1)               
tf.summary.histogram("W2", W2)               
tf.summary.histogram("W3", W3) 
tf.summary.histogram("b1", b1)                  
tf.summary.histogram("b2", b2)                  
tf.summary.histogram("b3", b3)    

#파라미터
training_epoch = 20
batch_size = 100

#세션런
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()                                   
    writer = tf.summary.FileWriter('./logs/MNN_3_Re_Xa_Dr', sess.graph)
    
    for epoch in range(training_epoch):
        avg_cost = 0    #
        total_batch =  int(mnist.train.num_examples / batch_size) 
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})    #
            avg_cost += c / total_batch
            
            summary = sess.run(merged, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 1})        #
            writer.add_summary(summary, global_step=sess.run(global_step))
            
        print('Epoch:', '%03d '%(epoch + 1),' Cost:', '%.4f'%(avg_cost), ' Acc:', 
              sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))     #    
#%%